In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar

class TrebuchetOptimizer:
    def __init__(self, m_missile, arm_length, frame_height, g=9.81):
        self.m_missile = m_missile
        self.arm_length = arm_length
        self.frame_height = frame_height
        self.g = g

    def launch_velocity(self, m_weight, arm_ratio, launch_angle):
        missile_arm_length = self.arm_length * (1 - arm_ratio)
        weight_arm_length = self.arm_length * arm_ratio

        # Потенциальная энергия противовеса
        h = self.frame_height + weight_arm_length * math.sin(launch_angle)
        potential_energy = m_weight * self.g * h
        
        # Кинетическая энергия снаряда
        velocity = math.sqrt((2 * potential_energy) / self.m_missile)
        return velocity

    def trajectory_distance(self, m_weight, arm_ratio, launch_angle, release_angle):
        v0 = self.launch_velocity(m_weight, arm_ratio, launch_angle)
        
        v_x = v0 * math.cos(release_angle)
        v_y = v0 * math.sin(release_angle)
        
        # Время до падения
        flight_time = (v_y + math.sqrt(v_y**2 + 2 * self.g * self.frame_height)) / self.g
        distance = v_x * flight_time
        return distance

    def optimize(self, arm_ratio_range=(0.1, 0.9), m_weight_guess=100, launch_angle_guess=math.pi/4):
        def objective(params):
            m_weight, arm_ratio, launch_angle, release_angle = params
            distance = -self.trajectory_distance(m_weight, arm_ratio, launch_angle, release_angle)
            return distance
        
        # Границы для параметров
        bounds = [
            (10, 1000),            # Масса противовеса
            (0.1, 0.9),            # Соотношение плеч рычага
            (0.1, math.pi/2),      # Угол запуска
            (0.1, math.pi/2)       # Угол вылета
        ]
        
        # Начальное приближение
        x0 = [m_weight_guess, 0.5, launch_angle_guess, math.pi/6]
        
        result = minimize_scalar(objective, method='L-BFGS-B', bounds=bounds)
        if result.success:
            return result.x  # Возвращаем оптимальные параметры
        else:
            return None

# Использование класса
trebuchet_optimizer = TrebuchetOptimizer(m_missile=10, arm_length=5, frame_height=2)
opt_params = trebuchet_optimizer.optimize()

if opt_params:
    m_weight, arm_ratio, launch_angle, release_angle = opt_params
    print(f"Оптимальные параметры:")
    print(f"Масса противовеса: {m_weight:.2f} кг")
    print(f"Соотношение плеч рычага: {arm_ratio:.2f}")
    print(f"Стартовый угол: {math.degrees(launch_angle):.2f} градусов")
    print(f"Угол вылета: {math.degrees(release_angle):.2f} градусов")
else:
    print("Оптимизация не удалась")


ValueError: Unknown solver L-BFGS-B